In [1]:
import sympy as sp

In [3]:
# declare a triangle element with 3 nodes in anticlockwise order
x_1, y_1 = sp.symbols('x_1 y_1')
x_2, y_2 = sp.symbols('x_2 y_2')
x_3, y_3 = sp.symbols('x_3 y_3')
x, y = sp.symbols('x y')
assump = sp.assumptions.AssumptionsContext()
jacobian = sp.Matrix([
    [1, x_1, y_1],
    [1, x_2, y_2],
    [1, x_3, y_3]
])
jacobian_det = jacobian.det()
assump.add(sp.Q.is_true(jacobian_det > 0))

$$
\begin{bmatrix}
x\\ y
\end{bmatrix}=
\begin{bmatrix}
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{bmatrix}
\begin{bmatrix}
\xi\\ \eta
\end{bmatrix}+
\begin{bmatrix}
b_1\\ b_2
\end{bmatrix}
$$

for $\xi,\eta\in[0,1]$, and tri's element's nodes: $(x_1,y_1),(x_2,y_2),(x_3,y_3)$, we have:

$$
\begin{bmatrix}
x_1 \\ y_1 \\
x_2 \\ y_2 \\
x_3 \\ y_3
\end{bmatrix}=
\begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1\\
1 & 0 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 1\\
0 & 1 & 1 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 1
\end{bmatrix}
\begin{bmatrix}
a_{11} \\ a_{12} \\ b_1 \\ a_{21} \\ a_{22} \\ b_2
\end{bmatrix}
$$

In [11]:
transfer_mat = sp.Matrix([
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 0, 1],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 1]
])
transfer_mat_inv = transfer_mat.inv()
transfer_mat_inv

Matrix([
[-1,  0, 1, 0, 0, 0],
[-1,  0, 0, 0, 1, 0],
[ 1,  0, 0, 0, 0, 0],
[ 0, -1, 0, 1, 0, 0],
[ 0, -1, 0, 0, 0, 1],
[ 0,  1, 0, 0, 0, 0]])

In [14]:
[a_11, a_12, b_1, a_21, a_22, b_2] = transfer_mat_inv @ sp.Matrix([x_1, y_1, x_2, y_2, x_3, y_3])
jacobi = sp.Matrix([
    [a_11, a_12],
    [a_21, a_22]
])
jacobi

Matrix([
[-x_1 + x_2, -x_1 + x_3],
[-y_1 + y_2, -y_1 + y_3]])

In [15]:
xi = sp.symbols('xi')
eta = sp.symbols('eta')

In [24]:
dxdy = jacobi.det()

In [25]:
dxdy

x_1*y_2 - x_1*y_3 - x_2*y_1 + x_2*y_3 + x_3*y_1 - x_3*y_2

In [18]:
xx = a_11 * xi + a_12 * eta + b_1
yy = a_21 * xi + a_22 * eta + b_2

In [27]:
mat_xi_eta = sp.Matrix([
    [1, 0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0, 0],
    [1, 0, 1, 0, 0, 1],
    [1, 0.5, 0, 0.25, 0, 0],
    [1, 0.5, 0.5, 0.25, 0.25, 0.25],
    [1, 0, 0.5, 0, 0, 0.25]
])
shape_func = sp.Matrix([1, xi, eta, xi ** 2, xi * eta, eta ** 2]).T @ mat_xi_eta.inv()

In [40]:
shape_func

Matrix([[2.0*eta**2 + 4.0*eta*xi - 3.0*eta + 2.0*xi**2 - 3.0*xi + 1, 2.0*xi**2 - 1.0*xi, 2.0*eta**2 - 1.0*eta, -4.0*eta*xi - 4.0*xi**2 + 4.0*xi, 4.0*eta*xi, -4.0*eta**2 - 4.0*eta*xi + 4.0*eta]])

In [34]:
shape_func.subs({xi: 0, eta: 0.5})

Matrix([[0, 0, 0, 0, 0, 1.0]])

In [37]:
mass_mat_nint = shape_func.T @ shape_func

In [38]:
mass_mat_int = sp.integrate(
    sp.integrate(mass_mat_nint, (xi, 0, 1 - eta)),
    (eta, 0, 1)
)

In [42]:
mass_mat_int

Matrix([
[  0.0166666666666675,  -0.00277777777777759,  -0.00277777777777785,   3.2612801348364e-15,   -0.0111111111111115,  4.44089209850063e-16],
[-0.00277777777777759,    0.0166666666666666,  -0.00277777777777771, -2.22044604925031e-16,  2.22044604925031e-16,    -0.011111111111111],
[-0.00277777777777785,  -0.00277777777777771,    0.0166666666666667,   -0.0111111111111109, -1.11022302462516e-16, -1.11022302462516e-16],
[ 3.2612801348364e-15, -2.22044604925031e-16,   -0.0111111111111109,    0.0888888888888892,     0.044444444444445,    0.0444444444444436],
[ -0.0111111111111115,  2.22044604925031e-16, -1.11022302462516e-16,     0.044444444444445,     0.088888888888889,    0.0444444444444445],
[4.44089209850063e-16,    -0.011111111111111, -1.11022302462516e-16,    0.0444444444444436,    0.0444444444444445,     0.088888888888889]])

In [43]:
import numpy as np

In [44]:
np.array(mass_mat_int).astype(np.float64)

array([[ 1.66666667e-02, -2.77777778e-03, -2.77777778e-03,
         3.26128013e-15, -1.11111111e-02,  4.44089210e-16],
       [-2.77777778e-03,  1.66666667e-02, -2.77777778e-03,
        -2.22044605e-16,  2.22044605e-16, -1.11111111e-02],
       [-2.77777778e-03, -2.77777778e-03,  1.66666667e-02,
        -1.11111111e-02, -1.11022302e-16, -1.11022302e-16],
       [ 3.26128013e-15, -2.22044605e-16, -1.11111111e-02,
         8.88888889e-02,  4.44444444e-02,  4.44444444e-02],
       [-1.11111111e-02,  2.22044605e-16, -1.11022302e-16,
         4.44444444e-02,  8.88888889e-02,  4.44444444e-02],
       [ 4.44089210e-16, -1.11111111e-02, -1.11022302e-16,
         4.44444444e-02,  4.44444444e-02,  8.88888889e-02]])

In [ ]:
mass_mat_int

In [23]:
sp.Matrix.jacobian(sp.Matrix([xx, yy]), sp.Matrix([xi, eta]))

Matrix([
[-x_1 + x_2, -x_1 + x_3],
[-y_1 + y_2, -y_1 + y_3]])

In [5]:
x_4 = (x_1 + x_2) / 2
y_4 = (y_1 + y_2) / 2
x_5 = (x_2 + x_3) / 2
y_5 = (y_2 + y_3) / 2
x_6 = (x_3 + x_1) / 2
y_6 = (y_3 + y_1) / 2

In [6]:
mat = sp.Matrix([
    [1, x_1, y_1, x_1**2, x_1*y_1, y_1**2],
    [1, x_2, y_2, x_2**2, x_2*y_2, y_2**2],
    [1, x_3, y_3, x_3**2, x_3*y_3, y_3**2],
    [1, x_4, y_4, x_4**2, x_4*y_4, y_4**2],
    [1, x_5, y_5, x_5**2, x_5*y_5, y_5**2],
    [1, x_6, y_6, x_6**2, x_6*y_6, y_6**2]
])

In [7]:
det = mat.det()

In [8]:
det

x_1**4*y_2**4/64 - x_1**4*y_2**3*y_3/16 + 3*x_1**4*y_2**2*y_3**2/32 - x_1**4*y_2*y_3**3/16 + x_1**4*y_3**4/64 - x_1**3*x_2*y_1*y_2**3/16 + 3*x_1**3*x_2*y_1*y_2**2*y_3/16 - 3*x_1**3*x_2*y_1*y_2*y_3**2/16 + x_1**3*x_2*y_1*y_3**3/16 + x_1**3*x_2*y_2**3*y_3/16 - 3*x_1**3*x_2*y_2**2*y_3**2/16 + 3*x_1**3*x_2*y_2*y_3**3/16 - x_1**3*x_2*y_3**4/16 + x_1**3*x_3*y_1*y_2**3/16 - 3*x_1**3*x_3*y_1*y_2**2*y_3/16 + 3*x_1**3*x_3*y_1*y_2*y_3**2/16 - x_1**3*x_3*y_1*y_3**3/16 - x_1**3*x_3*y_2**4/16 + 3*x_1**3*x_3*y_2**3*y_3/16 - 3*x_1**3*x_3*y_2**2*y_3**2/16 + x_1**3*x_3*y_2*y_3**3/16 + 3*x_1**2*x_2**2*y_1**2*y_2**2/32 - 3*x_1**2*x_2**2*y_1**2*y_2*y_3/16 + 3*x_1**2*x_2**2*y_1**2*y_3**2/32 - 3*x_1**2*x_2**2*y_1*y_2**2*y_3/16 + 3*x_1**2*x_2**2*y_1*y_2*y_3**2/8 - 3*x_1**2*x_2**2*y_1*y_3**3/16 + 3*x_1**2*x_2**2*y_2**2*y_3**2/32 - 3*x_1**2*x_2**2*y_2*y_3**3/16 + 3*x_1**2*x_2**2*y_3**4/32 - 3*x_1**2*x_2*x_3*y_1**2*y_2**2/16 + 3*x_1**2*x_2*x_3*y_1**2*y_2*y_3/8 - 3*x_1**2*x_2*x_3*y_1**2*y_3**2/16 + 3*x_1**2*x_2*x